<a href="https://colab.research.google.com/github/Luv4as/ia4good-rede-neural/blob/master/gnn-recommender-system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Sistema de Recomendação com Graph Neural Networks (GNN)

## 📋 Objetivo do Experimento

Este notebook implementa e compara diferentes arquiteturas de GNN para sistemas de recomendação:

1. **NGCF** (Neural Graph Collaborative Filtering) - Baseline
2. **LightGCN** - Versão simplificada do NGCF
3. **UltraGCN** - Estado da arte (SOTA)

## 🎯 Plano de Experimentos

1. **Experimento 1**: NGCF vs LightGCN no dataset Yelp
2. **Experimento 2**: NGCF vs LightGCN no dataset Amazon Books
3. **Experimento 3**: UltraGCN em ambos os datasets
4. **Análise Final**: Comparação completa dos resultados

## ⚙️ Configuração do Ambiente

**📖 Veja o arquivo `README.md` para instruções completas de instalação!**

- **Conda**: `conda env create -f environment.yml`
- **Pip**: `pip install -r requirements.txt`
- **Kaggle**: Upload direto (plug & play)

## 🔧 Configurações Otimizadas

- Datasets reduzidos com k-core filtering
- Hiperparâmetros ajustados para execução rápida
- Suporte automático para GPU quando disponível


In [ ]:
# Instalação das dependências (otimizada para Kaggle)
import subprocess
import sys


def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✓ {package} instalado com sucesso")
    except Exception as e:
        print(f"✗ Erro ao instalar {package}: {e}")


# Tentar instalar PyTorch Geometric com fallback
try:
    install_package("torch-geometric")
    install_package("torch-scatter")
    install_package("torch-sparse")
except Exception:
    print("Usando instalação alternativa...")
    install_package("torch-geometric==2.3.1")

print("Instalação concluída!")

✓ torch-geometric instalado com sucesso


In [ ]:
# Validações iniciais e configurações para Kaggle
import warnings
import os
import torch

warnings.filterwarnings("ignore")

# Verificar se estamos no Kaggle
is_kaggle = "KAGGLE_WORKING_DIR" in os.environ
print(f"Executando no Kaggle: {is_kaggle}")

# Configurações de memória para Kaggle
if is_kaggle:
    import gc

    gc.collect()
    print("Configurações de memória otimizadas para Kaggle")

print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(
        f"Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB"
    )

Executando no Kaggle: False
CUDA disponível: False


In [ ]:
# Importações principais
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

# PyTorch Geometric para os datasets
import torch_geometric
from torch_geometric.datasets import Yelp, AmazonBook

# Para construir a matriz de adjacência
from scipy.sparse import coo_matrix, hstack, vstack

import time
import gc


# --- Configurações Globais e Reprodutibilidade ---
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
print(f"Versão do PyTorch Geometric: {torch_geometric.__version__}")

Usando dispositivo: cpu
Versão do PyTorch Geometric: 2.6.1


# Preparação dos Dados


In [ ]:
# Converter os dados do PyG para DataFrames pandas
def pyg_to_dataframe(data):
    # Check if the data is HeteroData
    if isinstance(data, torch_geometric.data.HeteroData):
        # Assuming the user-item interaction edges are stored under ('user', 'rates', 'item')
        # This might need adjustment based on the specific dataset structure if different
        try:
            edge_index = data['user', 'rates', 'item'].edge_index.numpy()
            # For heterogeneous data, ratings might be stored differently or not at all.
            # Assuming implicit feedback (rating = 1) for now if not explicitly available.
            if hasattr(data['user', 'rates', 'item'], 'y') and data['user', 'rates', 'item'].y is not None:
                 ratings = data['user', 'rates', 'item'].y.numpy()
                 if ratings.ndim != 1:
                     ratings = ratings.flatten()
            else:
                 ratings = np.ones(edge_index.shape[1]) # Use shape[1] for number of edges
        except KeyError:
            # Handle cases where the edge type might be different
            print("Warning: Assuming edge type ('user', 'rates', 'item') not found. Trying other edge types.")
            # Attempt to find a user-item edge type - this is a heuristic and might need dataset-specific logic
            user_item_edge_type = None
            for edge_type in data.edge_types:
                 if 'user' in edge_type and 'item' in edge_type:
                      user_item_edge_type = edge_type
                      break

            if user_item_edge_type:
                 print(f"Using edge type: {user_item_edge_type}")
                 edge_index = data[user_item_edge_type].edge_index.numpy()
                 if hasattr(data[user_item_edge_type], 'y') and data[user_item_edge_type].y is not None:
                      ratings = data[user_item_edge_type].y.numpy()
                      if ratings.ndim != 1:
                          ratings = ratings.flatten()
                 else:
                      ratings = np.ones(edge_index.shape[1])
            else:
                 raise ValueError("Could not find a suitable user-item edge type in HeteroData.")


        user_ids = edge_index[0]
        item_ids = edge_index[1]

        # Ensure user_ids and item_ids are 1D (should be the case for edge_index)
        if user_ids.ndim != 1:
            user_ids = user_ids.flatten()
        if item_ids.ndim != 1:
            item_ids = item_ids.flatten()

        num_interactions = len(user_ids)

        # Ensure ratings has the same length as user_ids/item_ids
        if len(ratings) > num_interactions:
            ratings = ratings[:num_interactions]
        elif len(ratings) < num_interactions:
             print(f"Warning: Ratings array shorter than edge index ({len(ratings)} vs {num_interactions}). Padding with 1s.")
             ratings = np.concatenate([ratings, np.ones(num_interactions - len(ratings))])


    else: # Handle homogeneous Data
        edge_index = data.edge_index.numpy()
        user_ids = edge_index[0]
        item_ids = edge_index[1]

        # Ensure user_ids and item_ids are 1D
        if user_ids.ndim != 1:
            user_ids = user_ids.flatten()
        if item_ids.ndim != 1:
            item_ids = item_ids.flatten()

        num_interactions = len(user_ids)

        # Handle data.y potentially not existing or having a different shape
        if hasattr(data, 'y') and data.y is not None:
            ratings = data.y.numpy()
            if ratings.ndim != 1:
                ratings = ratings.flatten()
            # Ensure ratings has the same length as user_ids/item_ids
            if len(ratings) > num_interactions:
                ratings = ratings[:num_interactions]
            elif len(ratings) < num_interactions:
                 print(f"Warning: Ratings array shorter than edge index ({len(ratings)} vs {num_interactions}). Padding with 1s.")
                 ratings = np.concatenate([ratings, np.ones(num_interactions - len(ratings))])
        else:
            # Assume implicit feedback if no ratings are provided
            ratings = np.ones(num_interactions)

    # Final check on lengths before creating DataFrame
    if len(user_ids) != len(item_ids) or len(user_ids) != len(ratings):
         raise ValueError(f"Length mismatch: user_ids={len(user_ids)}, item_ids={len(item_ids)}, ratings={len(ratings)}")


    df = pd.DataFrame({"user_id": user_ids, "item_id": item_ids, "rating": ratings})
    return df


def filter_k_core(df, k=20):
    print(f"\nIniciando filtragem k-core com k={k}...")
    while True:
        # Contar interações por usuário e item
        user_counts = df["user_id"].value_counts()
        item_counts = df["item_id"].value_counts()

        # Encontrar usuários e itens com menos de K interações
        weak_users = user_counts[user_counts < k].index
        weak_items = item_counts[item_counts < k].index

        # Se não há mais ninguém para remover, o processo terminou
        if len(weak_users) == 0 and len(weak_items) == 0:
            print("Filtragem k-core concluída.")
            break

        # Remover as linhas com usuários ou itens fracos
        print(
            f"Removendo {len(weak_users)} usuários e {len(weak_items)} itens fracos..."
        )
        df = df[~df["user_id"].isin(weak_users)]
        df = df[~df["item_id"].isin(weak_items)]

    return df


# Reindexar IDs de usuários e itens para começarem em 0 e serem contínuos
def reindex_ids(df):
    print("\nReindexando IDs de usuários e itens...")
    user_id_mapping = {
        old_id: new_id for new_id, old_id in enumerate(df["user_id"].unique())
    }
    item_id_mapping = {
        old_id: new_id for new_id, old_id in enumerate(df["item_id"].unique())
    }

    df["user_id"] = df["user_id"].map(user_id_mapping)
    df["item_id"] = df["item_id"].map(item_id_mapping)

    num_users = len(user_id_mapping)
    num_items = len(item_id_mapping)
    print(f"Número de usuários: {num_users}, Número de itens: {num_items}")
    return df, num_users, num_items


# Dividir os dados em treino e teste
def train_test_split(df, test_size=0.2):
    print("\nDividindo os dados em treino e teste...")
    df = df.sample(frac=1, random_state=42).reset_index(
        drop=True
    )  # Embaralhar os dados
    test_indices = []
    train_indices = []

    user_group = df.groupby("user_id")
    for user_id, group in user_group:
        n_interactions = len(group)
        n_test = max(
            1, int(n_interactions * test_size)
        )  # Garantir pelo menos uma interação no teste
        test_indices.extend(group.index[:n_test])
        train_indices.extend(group.index[n_test:])

    test_df = df.loc[test_indices].reset_index(drop=True)
    train_df = df.loc[train_indices].reset_index(drop=True)

    print(
        f"Número de interações de treino: {len(train_df)}, Número de interações de teste: {len(test_df)}"
    )
    return train_df, test_df

In [ ]:
# Função para reduzir dataset (para execução mais rápida)
def reduce_dataset(df, max_users=5000, max_items=3000):
    """Reduz o dataset mantendo apenas os usuários e itens mais ativos"""
    print(f"Dataset original: {len(df)} interações")

    # Pegar os usuários mais ativos
    user_counts = df["user_id"].value_counts()
    top_users = user_counts.head(max_users).index

    # Filtrar por usuários mais ativos
    df_reduced = df[df["user_id"].isin(top_users)]

    # Pegar os itens mais populares
    item_counts = df_reduced["item_id"].value_counts()
    top_items = item_counts.head(max_items).index

    # Filtrar por itens mais populares
    df_reduced = df_reduced[df_reduced["item_id"].isin(top_items)]

    print(f"Dataset reduzido: {len(df_reduced)} interações")
    return df_reduced


# Carregar o dataset Yelp do PyG
print("\n=== CARREGANDO DATASET YELP ===")
yelp_dataset = Yelp(root="data/Yelp")
yelp_data = yelp_dataset[0]
print("Dataset Yelp carregado.")

# Carregar o dataset Amazon do PyG
print("\n=== CARREGANDO DATASET AMAZON BOOKS ===")
amazon_dataset = AmazonBook(root="data/AmazonBook")
amazon_data = amazon_dataset[0]
print("Dataset Amazon Books carregado.")

# Transformar os dados do PyG em DataFrames pandas
yelp_df = pyg_to_dataframe(yelp_data)
amazon_df = pyg_to_dataframe(amazon_data)
print("\nTransformação dos dados concluída.")

# Reduzir datasets para execução mais rápida
print("\n=== REDUZINDO DATASETS ===")
yelp_df = reduce_dataset(yelp_df, max_users=3000, max_items=2000)
amazon_df = reduce_dataset(amazon_df, max_users=3000, max_items=2000)

# Aplicar filtragem k-core (com k menor para datasets reduzidos)
print("\n=== APLICANDO FILTRAGEM K-CORE ===")
yelp_df = filter_k_core(yelp_df, k=5)  # k menor para datasets reduzidos
amazon_df = filter_k_core(amazon_df, k=5)

# Reindexar IDs de usuários e itens
print("\n=== REINDEXANDO IDs ===")
yelp_df, num_yelp_users, num_yelp_items = reindex_ids(yelp_df)
amazon_df, num_amazon_users, num_amazon_items = reindex_ids(amazon_df)

# Dividir os dados em treino e teste
print("\n=== DIVIDINDO TREINO/TESTE ===")
yelp_train_df, yelp_test_df = train_test_split(yelp_df, test_size=0.2)
amazon_train_df, amazon_test_df = train_test_split(amazon_df, test_size=0.2)

print("\n=== ESTATÍSTICAS FINAIS ===")
print(f"Yelp - Usuários: {num_yelp_users}, Itens: {num_yelp_items}")
print(f"Amazon - Usuários: {num_amazon_users}, Itens: {num_amazon_items}")
print("Pré-processamento concluído!")


=== CARREGANDO DATASET YELP ===
Dataset Yelp carregado.

=== CARREGANDO DATASET AMAZON BOOKS ===
Dataset Amazon Books carregado.


AttributeError: 'EdgeStorage' object has no attribute 'edge_index'

# Construção do Grafo e Matriz de Adjacência


In [ ]:
def get_adj_matrix(train_df, num_users, num_items):
    # Validar dados de entrada
    if len(train_df) == 0:
        raise ValueError("DataFrame de treino está vazio!")
    if num_users <= 0 or num_items <= 0:
        raise ValueError(
            f"Número inválido de usuários ({num_users}) ou itens ({num_items})"
        )

    # Criar uma matriz de adjacência esparsa (usuários x itens)
    adj_mat = coo_matrix(
        (np.ones(len(train_df)), (train_df["user_id"], train_df["item_id"])),
        shape=(num_users, num_items),
    )

    # Construir a matriz de adjacência completa (usuários+itens x usuários+itens)
    # R = [[0, A], [A.T, 0]]
    A = adj_mat
    A_t = A.transpose()

    # Criar blocos da matriz
    upper_left = coo_matrix((num_users, num_users))  # Zero block
    upper_right = A  # User-Item connections
    lower_left = A_t  # Item-User connections
    lower_right = coo_matrix((num_items, num_items))  # Zero block

    upper_block = hstack([upper_left, upper_right])
    lower_block = hstack([lower_left, lower_right])
    adj_full = vstack([upper_block, lower_block])

    # Converter para COO format para normalização
    adj_full = adj_full.tocoo()

    # Normalização da matriz de adjacência (D^-0.5 * A * D^-0.5)
    degree = np.array(adj_full.sum(axis=1)).flatten()
    degree[degree == 0] = 1  # Evitar divisão por zero

    # Criar matriz diagonal de graus
    d_inv_sqrt = np.power(degree, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.0

    # Aplicar normalização
    adj_full.data = adj_full.data * d_inv_sqrt[adj_full.row] * d_inv_sqrt[adj_full.col]

    # Converter para tensor esparso do PyTorch
    indices = torch.LongTensor(np.vstack((adj_full.row, adj_full.col)))
    values = torch.FloatTensor(adj_full.data)
    shape = torch.Size(adj_full.shape)

    adj_norm_sparse = torch.sparse_coo_tensor(indices, values, shape).to(device)

    return adj_norm_sparse


# Criar matrizes de adjacência para ambos os datasets
print("=== CRIANDO MATRIZES DE ADJACÊNCIA ===")
yelp_adj_matrix = get_adj_matrix(yelp_train_df, num_yelp_users, num_yelp_items)
amazon_adj_matrix = get_adj_matrix(amazon_train_df, num_amazon_users, num_amazon_items)
print("Matrizes de adjacência criadas para ambos os datasets.")

# Definição das Arquiteturas GNN


In [ ]:
class NGCF(nn.Module):
    # Implementação simplificada do NGCF
    def __init__(self, num_users, num_items, adj_matrix, embedding_dim=64, n_layers=2):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.adj_matrix = adj_matrix
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers

        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Camadas de transformação (a parte "complexa" do NGCF)
        self.W1 = nn.ModuleList(
            [nn.Linear(embedding_dim, embedding_dim) for _ in range(n_layers)]
        )
        self.W2 = nn.ModuleList(
            [nn.Linear(embedding_dim, embedding_dim) for _ in range(n_layers)]
        )

        # Inicialização dos embeddings
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, users, pos_items, neg_items):
        # Propagação no grafo
        embeddings = torch.cat(
            [self.user_embedding.weight, self.item_embedding.weight], dim=0
        )
        all_layer_embeddings = [embeddings]

        for i in range(self.n_layers):
            # Propagação
            propagated_embeddings = torch.sparse.mm(self.adj_matrix, embeddings)

            # Transformações (a parte que o LightGCN remove)
            sum_embeddings = self.W1[i](propagated_embeddings)
            bi_embeddings = self.W2[i](embeddings * propagated_embeddings)  # Interação

            embeddings = F.leaky_relu(sum_embeddings + bi_embeddings)
            all_layer_embeddings.append(embeddings)

        # Concatenação das camadas
        final_embeddings = torch.cat(all_layer_embeddings, dim=1)

        # Obter embeddings finais para o batch
        u_final = final_embeddings[users]
        pos_i_final = final_embeddings[pos_items + self.num_users]
        neg_i_final = final_embeddings[neg_items + self.num_users]

        return u_final, pos_i_final, neg_i_final

In [ ]:
class LightGCN(nn.Module):
    # Sua Modificação: NGCF simplificado
    def __init__(self, num_users, num_items, adj_matrix, embedding_dim=64, n_layers=2):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.adj_matrix = adj_matrix
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers

        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, users, pos_items, neg_items):
        embeddings = torch.cat(
            [self.user_embedding.weight, self.item_embedding.weight], dim=0
        )
        all_layer_embeddings = [embeddings]

        for _ in range(self.n_layers):
            # Apenas a propagação, sem W1, W2 ou ativações
            embeddings = torch.sparse.mm(self.adj_matrix, embeddings)
            all_layer_embeddings.append(embeddings)

        # Média das camadas
        final_embeddings = torch.mean(torch.stack(all_layer_embeddings, dim=0), dim=0)

        u_final = final_embeddings[users]
        pos_i_final = final_embeddings[pos_items + self.num_users]
        neg_i_final = final_embeddings[neg_items + self.num_users]

        return u_final, pos_i_final, neg_i_final

In [ ]:
class UltraGCN(nn.Module):
    def __init__(
        self,
        num_users,
        num_items,
        adj_matrix,
        embedding_dim=64,
        lambda_reg=1e-4,
        gamma=1e-4,
        beta=0.5,
    ):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.lambda_reg = lambda_reg
        self.gamma = gamma
        self.beta = beta

        # Embeddings iniciais
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Inicialização dos embeddings
        nn.init.normal_(self.user_embedding.weight, std=0.1)
        nn.init.normal_(self.item_embedding.weight, std=0.1)

        # Pré-computar graus para eficiência
        self.register_buffer("user_degrees", self._compute_user_degrees(adj_matrix))
        self.register_buffer("item_degrees", self._compute_item_degrees(adj_matrix))

    def _compute_user_degrees(self, adj_matrix):
        """Calcula os graus dos usuários a partir da matriz de adjacência"""
        # adj_matrix tem formato (num_users + num_items, num_users + num_items)
        # Pegar apenas a parte dos usuários (primeiras num_users linhas)
        adj_dense = adj_matrix.to_dense()
        user_part = adj_dense[: self.num_users, self.num_users :]  # usuários -> itens
        degrees = user_part.sum(dim=1)
        return degrees.float()

    def _compute_item_degrees(self, adj_matrix):
        """Calcula os graus dos itens a partir da matriz de adjacência"""
        adj_dense = adj_matrix.to_dense()
        item_part = adj_dense[self.num_users :, : self.num_users]  # itens -> usuários
        degrees = item_part.sum(dim=1)
        return degrees.float()

    def forward(self, users, pos_items, neg_items):
        """Forward pass simples - apenas lookup de embeddings"""
        u_embeds = self.user_embedding(users)
        pos_i_embeds = self.item_embedding(pos_items)
        neg_i_embeds = self.item_embedding(neg_items)
        return u_embeds, pos_i_embeds, neg_i_embeds

    def calculate_loss(self, users, pos_items, neg_items):
        """Calcula a loss completa do UltraGCN"""
        u_embeds, pos_i_embeds, neg_i_embeds = self.forward(users, pos_items, neg_items)

        # 1. BPR Loss básica
        pos_scores = (u_embeds * pos_i_embeds).sum(dim=1)
        neg_scores = (u_embeds * neg_i_embeds).sum(dim=1)
        bpr_loss = F.softplus(neg_scores - pos_scores).mean()

        # 2. Constraint Loss baseada nos graus dos nós
        # Pesos baseados nos graus (quanto maior o grau, maior o peso)
        user_deg_weights = torch.sqrt(self.user_degrees[users] + 1e-8)
        pos_item_deg_weights = torch.sqrt(self.item_degrees[pos_items] + 1e-8)
        neg_item_deg_weights = torch.sqrt(self.item_degrees[neg_items] + 1e-8)

        # Normalizar pesos
        user_deg_weights = user_deg_weights / (user_deg_weights.max() + 1e-8)
        pos_item_deg_weights = pos_item_deg_weights / (
            pos_item_deg_weights.max() + 1e-8
        )
        neg_item_deg_weights = neg_item_deg_weights / (
            neg_item_deg_weights.max() + 1e-8
        )

        # Constraint loss com pesos baseados em graus
        constraint_loss = self.lambda_reg * (
            (user_deg_weights * u_embeds.norm(p=2, dim=1)).mean()
            + (pos_item_deg_weights * pos_i_embeds.norm(p=2, dim=1)).mean()
            + (neg_item_deg_weights * neg_i_embeds.norm(p=2, dim=1)).mean()
        )

        # 3. Regularização L2 adicional
        l2_reg = self.gamma * (
            u_embeds.norm(p=2, dim=1).mean()
            + pos_i_embeds.norm(p=2, dim=1).mean()
            + neg_i_embeds.norm(p=2, dim=1).mean()
        )

        total_loss = bpr_loss + constraint_loss + l2_reg
        return total_loss

# Funções de Avaliação e Treinamento


In [ ]:
# Função para amostragem de negativos (corrigida)
def sample_negatives(train_df, num_items, num_samples=1):
    user_pos_items = train_df.groupby("user_id")["item_id"].apply(set).to_dict()
    neg_items = {}

    for user, pos_set in user_pos_items.items():
        neg_samples = []
        max_attempts = num_samples * len(pos_set) * 10  # Limite de tentativas
        attempts = 0

        while len(neg_samples) < num_samples * len(pos_set) and attempts < max_attempts:
            neg_id = random.randint(0, num_items - 1)
            if neg_id not in pos_set and neg_id not in neg_samples:
                neg_samples.append(neg_id)
            attempts += 1

        # Se não conseguir amostras suficientes, preencher com aleatórios
        while len(neg_samples) < num_samples * len(pos_set):
            neg_id = random.randint(0, num_items - 1)
            neg_samples.append(neg_id)

        neg_items[user] = neg_samples
    return neg_items


# Funções de Métrica simplificadas
def calculate_metrics(test_items, topk_items, k=20):
    """Calcula Recall@k e NDCG@k para um usuário"""
    hits = len(test_items.intersection(topk_items))

    if len(test_items) == 0:
        return 0.0, 0.0

    # Recall@k
    recall = hits / len(test_items)

    # NDCG@k simplificado
    dcg = sum(
        1.0 / np.log2(i + 2) for i, item in enumerate(topk_items) if item in test_items
    )
    idcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(test_items), k)))
    ndcg = dcg / idcg if idcg > 0 else 0.0

    return recall, ndcg


# Função de avaliação corrigida
def evaluate_model(model, test_df, train_df, num_users, num_items, k=20):
    model.eval()
    with torch.no_grad():
        # Obter embeddings finais de usuários e itens
        if isinstance(model, UltraGCN):
            # Para UltraGCN, usar embeddings diretamente
            all_user_embeds = model.user_embedding.weight
            all_item_embeds = model.item_embedding.weight
        else:
            # Para NGCF e LightGCN, computar embeddings finais
            # Usar embeddings completos depois da propagação
            embeddings = torch.cat(
                [model.user_embedding.weight, model.item_embedding.weight], dim=0
            )

            if isinstance(model, NGCF):
                # Para NGCF, aplicar as camadas de propagação
                all_layer_embeddings = [embeddings]
                for i in range(model.n_layers):
                    propagated_embeddings = torch.sparse.mm(
                        model.adj_matrix, embeddings
                    )
                    sum_embeddings = model.W1[i](propagated_embeddings)
                    bi_embeddings = model.W2[i](embeddings * propagated_embeddings)
                    embeddings = F.leaky_relu(sum_embeddings + bi_embeddings)
                    all_layer_embeddings.append(embeddings)
                final_embeddings = torch.cat(all_layer_embeddings, dim=1)
            else:
                # Para LightGCN, aplicar propagação simples
                all_layer_embeddings = [embeddings]
                for _ in range(model.n_layers):
                    embeddings = torch.sparse.mm(model.adj_matrix, embeddings)
                    all_layer_embeddings.append(embeddings)
                final_embeddings = torch.mean(
                    torch.stack(all_layer_embeddings, dim=0), dim=0
                )

            all_user_embeds = final_embeddings[:num_users]
            all_item_embeds = final_embeddings[num_users:]

    # Preparar dados para avaliação
    user_pos_items = train_df.groupby("user_id")["item_id"].apply(set).to_dict()
    test_user_pos_items = test_df.groupby("user_id")["item_id"].apply(set).to_dict()

    total_recall = 0
    total_ndcg = 0
    num_test_users = 0

    for user_id, test_items in tqdm(test_user_pos_items.items(), desc="Avaliando"):
        if user_id not in user_pos_items or user_id >= num_users:
            continue

        user_embed = all_user_embeds[user_id]
        scores = torch.matmul(user_embed, all_item_embeds.t())

        # Remover itens já vistos no treino
        train_items = user_pos_items[user_id]
        for item in train_items:
            if item < len(scores):
                scores[item] = -float("inf")

        # Pegar top-k itens
        _, topk_items = torch.topk(scores, min(k, len(scores)))
        topk_items_set = set(topk_items.cpu().numpy())

        if len(test_items) > 0:
            recall, ndcg = calculate_metrics(test_items, topk_items_set, k)
            total_recall += recall
            total_ndcg += ndcg

        num_test_users += 1

    if num_test_users == 0:
        return 0.0, 0.0

    return total_recall / num_test_users, total_ndcg / num_test_users


# BPR Loss
def bpr_loss(u_embeds, pos_i_embeds, neg_i_embeds):
    pos_scores = (u_embeds * pos_i_embeds).sum(dim=1)
    neg_scores = (u_embeds * neg_i_embeds).sum(dim=1)
    return F.softplus(neg_scores - pos_scores).mean()


# Função de treinamento genérica com batching (com validações)
def train_model(
    model,
    train_df,
    num_users,
    num_items,
    epochs=10,
    lr=0.001,
    batch_size=2048,
    model_name="Modelo",
):
    print(f"\n=== TREINANDO {model_name.upper()} ===")

    # Validações
    if len(train_df) == 0:
        raise ValueError("DataFrame de treino está vazio!")

    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Preparar dados para BPR
    train_users = train_df["user_id"].values
    train_pos_items = train_df["item_id"].values

    # Gerar amostras negativas
    try:
        neg_item_samples = sample_negatives(train_df, num_items, num_samples=1)
        train_neg_items = np.array([neg_item_samples[u][0] for u in train_users])
    except Exception as e:
        print(f"Erro na amostragem de negativos: {e}")
        raise

    # Criar índices para batching
    num_samples = len(train_users)
    indices = np.arange(num_samples)

    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        np.random.shuffle(indices)  # Embaralhar dados a cada época

        num_batches = (num_samples + batch_size - 1) // batch_size

        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]

            # Preparar batch
            batch_users = torch.LongTensor(train_users[batch_indices]).to(device)
            batch_pos_items = torch.LongTensor(train_pos_items[batch_indices]).to(
                device
            )
            batch_neg_items = torch.LongTensor(train_neg_items[batch_indices]).to(
                device
            )

            optimizer.zero_grad()

            if hasattr(model, "calculate_loss"):  # UltraGCN
                loss = model.calculate_loss(
                    batch_users, batch_pos_items, batch_neg_items
                )
            else:  # NGCF e LightGCN
                u_embeds, pos_i_embeds, neg_i_embeds = model(
                    batch_users, batch_pos_items, batch_neg_items
                )
                loss = bpr_loss(u_embeds, pos_i_embeds, neg_i_embeds)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / num_batches
        if (epoch + 1) % 2 == 0:
            print(f"{model_name} - Época {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    training_time = time.time() - start_time
    print(f"{model_name} treinado em {training_time:.2f}s")

    return training_time

# Execução do Experimento


In [ ]:
# Hiperparâmetros otimizados para Kaggle
EPOCHS = 3  # Reduzido para execução mais rápida no Kaggle
BATCH_SIZE = 1024  # Reduzido para evitar problemas de memória
EMBEDDING_DIM = 32  # Reduzido para acelerar treinamento
LEARNING_RATE = 0.001
N_LAYERS = 2  # Para NGCF e LightGCN

print("=== CONFIGURAÇÕES DO EXPERIMENTO ===")
print(f"Épocas: {EPOCHS}")
print(f"Tamanho do batch: {BATCH_SIZE}")
print(f"Dimensão dos embeddings: {EMBEDDING_DIM}")
print(f"Taxa de aprendizado: {LEARNING_RATE}")
print(f"Número de camadas GNN: {N_LAYERS}")
print(f"Dispositivo: {device}")
print("\n⚠️  Configurações otimizadas para execução rápida no Kaggle")

In [ ]:
# Verificação de integridade dos dados antes dos experimentos
print("=== VERIFICAÇÃO DE INTEGRIDADE DOS DADOS ===")


def verify_data_integrity():
    """Verifica se os dados estão corretos para os experimentos"""
    checks_passed = 0
    total_checks = 6

    # Check 1: DataFrames não vazios
    if len(yelp_train_df) > 0 and len(yelp_test_df) > 0:
        print("✓ Yelp: Dados de treino e teste não estão vazios")
        checks_passed += 1
    else:
        print("✗ Yelp: Dados vazios!")

    # Check 2: Amazon DataFrames não vazios
    if len(amazon_train_df) > 0 and len(amazon_test_df) > 0:
        print("✓ Amazon: Dados de treino e teste não estão vazios")
        checks_passed += 1
    else:
        print("✗ Amazon: Dados vazios!")

    # Check 3: IDs válidos no Yelp
    if (
        yelp_train_df["user_id"].max() < num_yelp_users
        and yelp_train_df["item_id"].max() < num_yelp_items
    ):
        print("✓ Yelp: IDs de usuários e itens estão dentro dos limites")
        checks_passed += 1
    else:
        print("✗ Yelp: IDs fora dos limites!")

    # Check 4: IDs válidos no Amazon
    if (
        amazon_train_df["user_id"].max() < num_amazon_users
        and amazon_train_df["item_id"].max() < num_amazon_items
    ):
        print("✓ Amazon: IDs de usuários e itens estão dentro dos limites")
        checks_passed += 1
    else:
        print("✗ Amazon: IDs fora dos limites!")

    # Check 5: Matrizes de adjacência criadas
    if yelp_adj_matrix is not None and amazon_adj_matrix is not None:
        print("✓ Matrizes de adjacência criadas com sucesso")
        checks_passed += 1
    else:
        print("✗ Erro na criação das matrizes de adjacência!")

    # Check 6: Device disponível
    if device.type in ["cuda", "cpu"]:
        print(f"✓ Dispositivo {device} disponível para computação")
        checks_passed += 1
    else:
        print("✗ Erro na configuração do dispositivo!")

    print(f"\nResumo: {checks_passed}/{total_checks} verificações passaram")

    if checks_passed == total_checks:
        print("🎉 Todos os dados estão íntegros! Pronto para experimentos.")
        return True
    else:
        print("⚠️  Alguns problemas detectados. Verifique os dados antes de continuar.")
        return False


# Executar verificação
data_ok = verify_data_integrity()

In [ ]:
# Verificar se os dados estão OK antes de executar experimentos
if not data_ok:
    print("❌ EXPERIMENTOS CANCELADOS!")
    print("Por favor, verifique os problemas nos dados acima antes de continuar.")
    print("Possíveis soluções:")
    print("- Reinstalar dependências")
    print("- Verificar conexão com internet para download dos datasets")
    print("- Reiniciar o kernel do notebook")
else:
    print("✅ Dados verificados! Iniciando experimentos...")
    print("🚀 Execute as próximas células para rodar os experimentos.")

In [ ]:
# =============================================================================
# EXPERIMENTO 1: NGCF vs LightGCN no Dataset YELP
# =============================================================================

if not data_ok:
    print("❌ Pulando experimento 1 devido a problemas nos dados!")
else:
    print("\n" + "=" * 80)
    print("EXPERIMENTO 1: NGCF vs LightGCN no Dataset YELP")
    print("=" * 80)

    # --- Modelo 1: NGCF no Yelp ---
    model_ngcf_yelp = NGCF(
        num_yelp_users, num_yelp_items, yelp_adj_matrix, EMBEDDING_DIM, N_LAYERS
    ).to(device)

    time_ngcf_yelp = train_model(
        model_ngcf_yelp,
        yelp_train_df,
        num_yelp_users,
        num_yelp_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="NGCF (Yelp)",
    )

    recall_ngcf_yelp, ndcg_ngcf_yelp = evaluate_model(
        model_ngcf_yelp, yelp_test_df, yelp_train_df, num_yelp_users, num_yelp_items
    )
    print(
        f"NGCF (Yelp) - Recall@20: {recall_ngcf_yelp:.4f}, NDCG@20: {ndcg_ngcf_yelp:.4f}"
    )

    # --- Modelo 2: LightGCN no Yelp ---
    model_lightgcn_yelp = LightGCN(
        num_yelp_users, num_yelp_items, yelp_adj_matrix, EMBEDDING_DIM, N_LAYERS
    ).to(device)

    time_lightgcn_yelp = train_model(
        model_lightgcn_yelp,
        yelp_train_df,
        num_yelp_users,
        num_yelp_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="LightGCN (Yelp)",
    )

    recall_lightgcn_yelp, ndcg_lightgcn_yelp = evaluate_model(
        model_lightgcn_yelp, yelp_test_df, yelp_train_df, num_yelp_users, num_yelp_items
    )
    print(
        f"LightGCN (Yelp) - Recall@20: {recall_lightgcn_yelp:.4f}, NDCG@20: {ndcg_lightgcn_yelp:.4f}"
    )

    print("\nResumo Yelp:")
    print(
        f"NGCF: Recall={recall_ngcf_yelp:.4f}, NDCG={ndcg_ngcf_yelp:.4f}, Tempo={time_ngcf_yelp:.2f}s"
    )
    print(
        f"LightGCN: Recall={recall_lightgcn_yelp:.4f}, NDCG={ndcg_lightgcn_yelp:.4f}, Tempo={time_lightgcn_yelp:.2f}s"
    )

In [ ]:
# =============================================================================
# EXPERIMENTO 2: NGCF vs LightGCN no Dataset AMAZON
# =============================================================================

if not data_ok:
    print("❌ Pulando experimento 2 devido a problemas nos dados!")
else:
    print("\n" + "=" * 80)
    print("EXPERIMENTO 2: NGCF vs LightGCN no Dataset AMAZON")
    print("=" * 80)

    # --- Modelo 1: NGCF no Amazon ---
    model_ngcf_amazon = NGCF(
        num_amazon_users, num_amazon_items, amazon_adj_matrix, EMBEDDING_DIM, N_LAYERS
    ).to(device)

    time_ngcf_amazon = train_model(
        model_ngcf_amazon,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="NGCF (Amazon)",
    )

    recall_ngcf_amazon, ndcg_ngcf_amazon = evaluate_model(
        model_ngcf_amazon,
        amazon_test_df,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
    )
    print(
        f"NGCF (Amazon) - Recall@20: {recall_ngcf_amazon:.4f}, NDCG@20: {ndcg_ngcf_amazon:.4f}"
    )

    # --- Modelo 2: LightGCN no Amazon ---
    model_lightgcn_amazon = LightGCN(
        num_amazon_users, num_amazon_items, amazon_adj_matrix, EMBEDDING_DIM, N_LAYERS
    ).to(device)

    time_lightgcn_amazon = train_model(
        model_lightgcn_amazon,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="LightGCN (Amazon)",
    )

    recall_lightgcn_amazon, ndcg_lightgcn_amazon = evaluate_model(
        model_lightgcn_amazon,
        amazon_test_df,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
    )
    print(
        f"LightGCN (Amazon) - Recall@20: {recall_lightgcn_amazon:.4f}, NDCG@20: {ndcg_lightgcn_amazon:.4f}"
    )

    print("\nResumo Amazon:")
    print(
        f"NGCF: Recall={recall_ngcf_amazon:.4f}, NDCG={ndcg_ngcf_amazon:.4f}, Tempo={time_ngcf_amazon:.2f}s"
    )
    print(
        f"LightGCN: Recall={recall_lightgcn_amazon:.4f}, NDCG={ndcg_lightgcn_amazon:.4f}, Tempo={time_lightgcn_amazon:.2f}s"
    )

In [ ]:
# =============================================================================
# EXPERIMENTO 3: UltraGCN nos Datasets YELP e AMAZON
# =============================================================================

if not data_ok:
    print("❌ Pulando experimento 3 devido a problemas nos dados!")
else:
    print("\n" + "=" * 80)
    print("EXPERIMENTO 3: UltraGCN nos Datasets YELP e AMAZON")
    print("=" * 80)

    # --- UltraGCN no Yelp ---
    model_ultragcn_yelp = UltraGCN(
        num_yelp_users, num_yelp_items, yelp_adj_matrix, EMBEDDING_DIM
    ).to(device)

    time_ultragcn_yelp = train_model(
        model_ultragcn_yelp,
        yelp_train_df,
        num_yelp_users,
        num_yelp_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="UltraGCN (Yelp)",
    )

    recall_ultragcn_yelp, ndcg_ultragcn_yelp = evaluate_model(
        model_ultragcn_yelp, yelp_test_df, yelp_train_df, num_yelp_users, num_yelp_items
    )
    print(
        f"UltraGCN (Yelp) - Recall@20: {recall_ultragcn_yelp:.4f}, NDCG@20: {ndcg_ultragcn_yelp:.4f}"
    )

    # --- UltraGCN no Amazon ---
    model_ultragcn_amazon = UltraGCN(
        num_amazon_users, num_amazon_items, amazon_adj_matrix, EMBEDDING_DIM
    ).to(device)

    time_ultragcn_amazon = train_model(
        model_ultragcn_amazon,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
        epochs=EPOCHS,
        lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        model_name="UltraGCN (Amazon)",
    )

    recall_ultragcn_amazon, ndcg_ultragcn_amazon = evaluate_model(
        model_ultragcn_amazon,
        amazon_test_df,
        amazon_train_df,
        num_amazon_users,
        num_amazon_items,
    )
    print(
        f"UltraGCN (Amazon) - Recall@20: {recall_ultragcn_amazon:.4f}, NDCG@20: {ndcg_ultragcn_amazon:.4f}"
    )

    print("\nResumo UltraGCN:")
    print(
        f"Yelp: Recall={recall_ultragcn_yelp:.4f}, NDCG={ndcg_ultragcn_yelp:.4f}, Tempo={time_ultragcn_yelp:.2f}s"
    )
    print(
        f"Amazon: Recall={recall_ultragcn_amazon:.4f}, NDCG={ndcg_ultragcn_amazon:.4f}, Tempo={time_ultragcn_amazon:.2f}s"
    )

In [ ]:
# =============================================================================
# RESUMO FINAL: COMPARAÇÃO DE TODOS OS RESULTADOS
# =============================================================================

if not data_ok:
    print("❌ Não é possível gerar resumo devido a problemas nos dados!")
else:
    print("\n" + "=" * 80)
    print("RESUMO FINAL - COMPARAÇÃO COMPLETA")
    print("=" * 80)

    # Criar tabela de resultados completa
    results_data = {
        "Modelo": ["NGCF", "LightGCN", "NGCF", "LightGCN", "UltraGCN", "UltraGCN"],
        "Dataset": ["Yelp", "Yelp", "Amazon", "Amazon", "Yelp", "Amazon"],
        "Recall@20": [
            recall_ngcf_yelp,
            recall_lightgcn_yelp,
            recall_ngcf_amazon,
            recall_lightgcn_amazon,
            recall_ultragcn_yelp,
            recall_ultragcn_amazon,
        ],
        "NDCG@20": [
            ndcg_ngcf_yelp,
            ndcg_lightgcn_yelp,
            ndcg_ngcf_amazon,
            ndcg_lightgcn_amazon,
            ndcg_ultragcn_yelp,
            ndcg_ultragcn_amazon,
        ],
        "Tempo (s)": [
            time_ngcf_yelp,
            time_lightgcn_yelp,
            time_ngcf_amazon,
            time_lightgcn_amazon,
            time_ultragcn_yelp,
            time_ultragcn_amazon,
        ],
    }

    results_df = pd.DataFrame(results_data)
    print("\n📊 TABELA DE RESULTADOS FINAIS:")
    print(results_df.round(4))

    # Análise comparativa
    print("\n📈 ANÁLISE COMPARATIVA:")
    print("\n1. NGCF vs LightGCN:")
    yelp_ngcf_vs_light = recall_lightgcn_yelp - recall_ngcf_yelp
    amazon_ngcf_vs_light = recall_lightgcn_amazon - recall_ngcf_amazon
    print(f"   • Yelp: LightGCN supera NGCF em {yelp_ngcf_vs_light:+.4f} (Recall@20)")
    print(
        f"   • Amazon: LightGCN supera NGCF em {amazon_ngcf_vs_light:+.4f} (Recall@20)"
    )

    print("\n2. Melhor modelo por dataset:")
    if recall_ultragcn_yelp > max(recall_ngcf_yelp, recall_lightgcn_yelp):
        print(f"   • Yelp: UltraGCN ({recall_ultragcn_yelp:.4f})")
    else:
        best_yelp = "LightGCN" if recall_lightgcn_yelp > recall_ngcf_yelp else "NGCF"
        best_yelp_score = max(recall_lightgcn_yelp, recall_ngcf_yelp)
        print(f"   • Yelp: {best_yelp} ({best_yelp_score:.4f})")

    if recall_ultragcn_amazon > max(recall_ngcf_amazon, recall_lightgcn_amazon):
        print(f"   • Amazon: UltraGCN ({recall_ultragcn_amazon:.4f})")
    else:
        best_amazon = (
            "LightGCN" if recall_lightgcn_amazon > recall_ngcf_amazon else "NGCF"
        )
        best_amazon_score = max(recall_lightgcn_amazon, recall_ngcf_amazon)
        print(f"   • Amazon: {best_amazon} ({best_amazon_score:.4f})")

    print("\n3. Tempo de treinamento:")
    avg_time_ngcf = (time_ngcf_yelp + time_ngcf_amazon) / 2
    avg_time_lightgcn = (time_lightgcn_yelp + time_lightgcn_amazon) / 2
    avg_time_ultragcn = (time_ultragcn_yelp + time_ultragcn_amazon) / 2

    print(f"   • NGCF: {avg_time_ngcf:.2f}s (média)")
    print(f"   • LightGCN: {avg_time_lightgcn:.2f}s (média)")
    print(f"   • UltraGCN: {avg_time_ultragcn:.2f}s (média)")

    print("\n✅ EXPERIMENTO CONCLUÍDO!")
    print("=" * 80)

## 🎯 Conclusões e Próximos Passos

### 📊 O que foi implementado:

- ✅ **3 arquiteturas GNN**: NGCF, LightGCN e UltraGCN
- ✅ **2 datasets**: Yelp e Amazon Books com pré-processamento
- ✅ **Métricas robustas**: Recall@20 e NDCG@20
- ✅ **Otimizações**: Batching, validações e configurações para Kaggle

### 🔬 Resultados esperados:

- **LightGCN** deve superar **NGCF** em eficiência e simplicidade
- **UltraGCN** deve apresentar os melhores resultados de precisão
- Datasets maiores (Amazon) podem favorecer arquiteturas mais complexas

### 🚀 Próximos passos:

1. Experimentos com hiperparâmetros diferentes
2. Testes em datasets maiores
3. Implementação de outras arquiteturas (LightGCL, NGCL)
4. Análise mais profunda dos embeddings aprendidos

### 📝 Para usar este notebook:

- **Kaggle**: Upload direto e execute
- **Local**: Siga as instruções no README.md
- **Customização**: Ajuste hiperparâmetros na seção correspondente


In [ ]:
# =============================================================================
# LIMPEZA FINAL E INFORMAÇÕES DO SISTEMA
# =============================================================================

# Tentar importar psutil, se não estiver disponível, usar alternativa
try:
    import psutil

    psutil_available = True
except ImportError:
    psutil_available = False
    print("⚠️  psutil não disponível - algumas informações de sistema serão limitadas")

print("=== LIMPEZA DE MEMÓRIA ===")

# Limpeza de memória
if "model_ngcf_yelp" in locals():
    del model_ngcf_yelp
if "model_lightgcn_yelp" in locals():
    del model_lightgcn_yelp
if "model_ultragcn_yelp" in locals():
    del model_ultragcn_yelp
if "model_ngcf_amazon" in locals():
    del model_ngcf_amazon
if "model_lightgcn_amazon" in locals():
    del model_lightgcn_amazon
if "model_ultragcn_amazon" in locals():
    del model_ultragcn_amazon

# Limpar cache do PyTorch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ Cache GPU limpo")

# Garbage collection
gc.collect()
print("✓ Garbage collection executado")

# Informações do sistema
print("\n=== INFORMAÇÕES DO SISTEMA ===")
print(f"Versão Python: {sys.version.split()[0]}")
print(f"Versão PyTorch: {torch.__version__}")
print(f"Versão PyTorch Geometric: {torch_geometric.__version__}")
print(f"Dispositivo usado: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(
        f"Memória GPU total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB"
    )
    print(
        f"Memória GPU livre: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()) / 1e9:.1f} GB"
    )

# Informações de memória RAM
if psutil_available:
    memory_info = psutil.virtual_memory()
    print(f"RAM total: {memory_info.total / 1e9:.1f} GB")
    print(f"RAM disponível: {memory_info.available / 1e9:.1f} GB")
    print(f"RAM em uso: {memory_info.percent:.1f}%")
else:
    print("RAM: Informações não disponíveis (psutil não instalado)")

print("\n✅ Experimento finalizado com sucesso!")
print("📁 Arquivos criados: README.md, environment.yml, requirements.txt, .gitignore")
print("🎉 Pronto para compartilhar ou continuar desenvolvimento!")

print("\n" + "=" * 60)
print("🚀 OBRIGADO POR USAR O GNN RECOMMENDER SYSTEM! 🚀")
print("=" * 60)